# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [41]:
import numpy as np
import os
#from scipy.misc import imread, imresize -> As imread and imresize are deprecated, we have used Pillow package
from PIL import Image
import imageio
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [42]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [24]:
## If you are using the data by mounting the google drive, use the following :
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
!unzip gdrive/MyDrive/Project_data.zip

Streaming output truncated to the last 5000 lines.
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00050.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00052.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00054.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00056.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00058.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00060.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00062.png  
   creating: Project_data/train/WIN_20180926_17_35_12_Pro_Thumbs_Down_new/
 extracting: Project_data/train/WIN_20180926_17_35_12_Pro_Thumbs_Down_new/WIN_20180926_17_35_12_Pro_00001.png  
 extracting: Project_data/t

In [67]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 30 #experiment with the batch size
x = 15 #number of imgaes to analyse per. video
(y,z) = (84,84)

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [88]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    #img_idx = [1,2,3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,20,21,23,24,25,27,28,29]#create a list of image numbers you want to use for a particular video
    img_idx = [0,2,5,7,9,10,12,15,18,20,21,23,25,27,29]
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list) // batch_size# calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    with Image.open(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]) as im:
                      im_resized = im.resize((y, z))
                    image = np.asarray(im_resized).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    norm_image = (image - np.min(image))/(np.max(image) - np.min(image))
                    batch_data[folder,idx,:,:,0] = norm_image[:,:,0]#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = norm_image[:,:,1]#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = norm_image[:,:,2]#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        remaining_seq = len(t) % batch_size
        if remaining_seq != 0:
          batch_data = np.zeros((remaining_seq,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
          batch_labels = np.zeros((remaining_seq,5)) # batch_labels is the one hot representation of the output
          for folder in range(remaining_seq): # iterate over the batch_size
              imgs = os.listdir(source_path+'/'+ t[folder + ((batch+1)*batch_size)].split(';')[0]) # read all the images in the folder
              for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                  with Image.open(source_path+'/'+ t[folder + ((batch+1)*batch_size)].strip().split(';')[0]+'/'+imgs[item]) as im:
                    im_resized = im.resize((y, z))
                  image = np.asarray(im_resized).astype(np.float32)
                      
                  #crop the images and resize them. Note that the images are of 2 different shape 
                  #and the conv3D will throw error if the inputs in a batch have different shapes
                  norm_image = (image - np.min(image))/(np.max(image) - np.min(image))
                  batch_data[folder,idx,:,:,0] = norm_image[:,:,0]#normalise and feed in the image
                  batch_data[folder,idx,:,:,1] = norm_image[:,:,1]#normalise and feed in the image
                  batch_data[folder,idx,:,:,2] = norm_image[:,:,2]#normalise and feed in the image
                      
              batch_labels[folder, int(t[folder + (batch*remaining_seq)].strip().split(';')[2])] = 1
          yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do



Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [89]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 25# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 25


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [90]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from tensorflow.keras import regularizers

#write your model here
num_classes = 5

model = Sequential()
model.add(Conv3D(32, (3, 3,3), input_shape = (x,y,z,3)))
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2), padding='same'))
model.add(Dropout(0.2))

model.add(Conv3D(32, (3, 3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2), padding='same'))

model.add(Conv3D(32, (3, 3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2), padding='same'))

#model.add(Conv3D(64, (3, 3, 3)))
#model.add(Activation('relu'))
#model.add(BatchNormalization())
#model.add(MaxPooling3D(pool_size=(2, 2, 2), padding='same'))
model.add(Dropout(0.2))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(512,kernel_regularizer=tf.keras.regularizers.l2(0.04)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [91]:
optimiser = tf.keras.optimizers.RMSprop(learning_rate=0.001)#write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_50 (Conv3D)           (None, 13, 82, 82, 32)    2624      
_________________________________________________________________
activation_67 (Activation)   (None, 13, 82, 82, 32)    0         
_________________________________________________________________
max_pooling3d_45 (MaxPooling (None, 7, 41, 41, 32)     0         
_________________________________________________________________
dropout_39 (Dropout)         (None, 7, 41, 41, 32)     0         
_________________________________________________________________
conv3d_51 (Conv3D)           (None, 5, 39, 39, 32)     27680     
_________________________________________________________________
activation_68 (Activation)   (None, 5, 39, 39, 32)     0         
_________________________________________________________________
max_pooling3d_46 (MaxPooling (None, 3, 20, 20, 32)   

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [92]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [93]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', save_freq='epoch')

LR = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1,mode='auto')# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [94]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [95]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 30
Epoch 1/25
23/23 [==============================] - 31s 1s/step - loss: 21.7373 - categorical_accuracy: 0.1689 - val_loss: 5.0178 - val_categorical_accuracy: 0.1900

Epoch 00001: saving model to model_init_2021-04-1916_59_45.068303/model-00001-14.85616-0.19005-5.01778-0.19000.h5
Epoch 2/25
23/23 [==============================] - 30s 1s/step - loss: 3.7274 - categorical_accuracy: 0.2363 - val_loss: 1.8652 - val_categorical_accuracy: 0.1600

Epoch 00002: saving model to model_init_2021-04-1916_59_45.068303/model-00002-3.00193-0.24284-1.86520-0.16000.h5
Epoch 3/25
23/23 [==============================] - 30s 1s/step - loss: 1.7515 - categorical_accuracy: 0.2363 - val_loss: 1.6287 - val_categorical_accuracy: 0.2100

Epoch 00003: saving model to model_init_2021-04-1916_59_45.068303/model-00003-1.68263-0.26546-1.62869-0.21000.h5
Epoch 4/25
23/23 [==============================] - 29s 1s/step - loss: 1.6226 - categorical_accuracy: 0.3273 - 